In [1]:
%load_ext autoreload
%autoreload 2
%cd "python/LatentEvolution"
%ls

/home/hew/python/LatentEvolution
cache/  data/  figure/  framework/  main.py  script/  temp/


In [2]:
import gc

import torch

from framework.config import parse_config, paths
from framework.utils.lightning.device_utils import seed_everything
from framework.utils.lightning.trainer_utils import get_pl_trainer
from script.task_04_LatentRegression.LatentRegression.sequence_data_module import SequenceDataModule
from script.task_04_LatentRegression.LatentRegression.sequence_lightning_module import SequenceLightningModule

========== add root_path and framework_path to sys.path ==========
root_path: /home/hew/python/LatentEvolution
framework_path: /home/hew/python/LatentEvolution/framework


In [3]:
update_dict = {
    'project': 'LatentRegression',
    'seed': 42,
    'logger': {
        'save_dir': './script/task_04_LatentRegression/'
    },
    'data': {
        'dataset': 'ACE2_latent_variants',
        'data_class': 'Protein',
        'mini_set_ratio': None,
        'max_len': 83,
    },
    'hparams': {
        'mlp_params': {
            # 'hiddens': [128, 64, 32, 2],
            'hiddens': [1360, 256, 2],
            'activation': 'ReLU',
            'batch_norm': False,
            'bias': True,
            'dropout': 0.,
        },
    },
    'loss': {
        'mse_loss': {'name': 'MSELoss', 'args': {}},
    },
    'optimizer': {
        'name': 'Adam',
        # 'name': 'RAdam',
        # 'name': 'AdamW',
        'args': {
            # 'lr': 0.001,
            'lr': 0.0005,
            # 'lr': 0.0002,
            # 'lr': 0.0001,
        }
    },
    'scheduler': {
        # 'name': None,
        # 'args': {},
        'name': 'LinearLR',
        'interval': 'step',
        'frequency': 1,
        'args': {
            'start_factor': 1,
            'end_factor': 0.1,
            'total_iters': 25 * 50
        },
    },
    'trainer': {
        'max_epochs': 50,
        'gradient_clip_val': 1.0,
        'gradient_clip_algorithm': 'norm',
        'accumulate_grad_batches': 1,
        'num_sanity_val_steps': 2,
        'val_check_interval': 0.5,
        'enable_checkpointing': True,
        # 'enable_checkpointing': False,
    },
    'early_stop_callback': {
        'monitor': 'valid/loss_epoch',
        'mode': 'min',
    },
    'ckpt_callback': {
        'monitor': 'valid/loss_epoch',
        'filename': 'epoch={epoch:02d}, loss={valid/loss_epoch:.3f}, ddG={valid/ddG_pearsonr_epoch:.3f}, dS={valid/dS_pearsonr_epoch:.3f}, avg={valid/avg_pearsonr_epoch:.3f}',
        'auto_insert_metric_name': False,
        'save_weights_only': True,
        'mode': 'min',
        'save_top_k': 50,
        'save_last': True,
    },
    # 'ckpt_callback': None,
    'train_dataloader': {
        # 'batch_size': 128,
        'batch_size': 64,
        'num_workers': 4,
        'drop_last': True,
    },
    'valid_dataloader': {
        # 'batch_size': 128,
        'batch_size': 64,
        'num_workers': 4,
    },
    'test_dataloader': {
        # 'batch_size': 128,
        'batch_size': 64,
        'num_workers': 4,
    },
    'predict_dataloader': {
        # 'batch_size': 128,
        'batch_size': 64,
        'num_workers': 4,
    },
}
dataset_hparams = paths.script + '/task_04_LatentRegression/LatentRegression/dataset.yaml'
model_hparams = paths.script + '/task_04_LatentRegression/LatentRegression/model.yaml'
framework_hparams = paths.script + '/task_04_LatentRegression/LatentRegression/framework.yaml'
args = parse_config([dataset_hparams, model_hparams, framework_hparams, update_dict])

In [4]:
args

{'project': 'LatentRegression', 'dataset': 'template', 'model': '/home/hew/python/LatentEvolution/framework/config/model/template.yaml', 'seed': 42, 'tokenization': {'alphabet': 'ESM-1b', 'truncation_seq_length': None}, 'train_dataloader': {'batch_size': 64, 'num_workers': 4, 'shuffle': True, 'drop_last': True, 'pin_memory': True, 'persistent_workers': True}, 'valid_dataloader': {'batch_size': 64, 'num_workers': 4, 'shuffle': False, 'pin_memory': True, 'persistent_workers': True}, 'test_dataloader': {'batch_size': 64, 'num_workers': 4, 'shuffle': False, 'pin_memory': True, 'persistent_workers': True}, 'predict_dataloader': {'batch_size': 64, 'num_workers': 4, 'shuffle': False, 'pin_memory': True, 'persistent_workers': True}, 'trainer': {'max_epochs': 50, 'accelerator': 'auto', 'strategy': 'auto', 'devices': 'auto', 'deterministic': False, 'benchmark': True, 'sync_batchnorm': True, 'log_every_n_steps': 1, 'check_val_every_n_epoch': 1, 'fast_dev_run': False, 'num_sanity_val_steps': 2, 'e

In [5]:
%%time
seed_everything(args.seed)
pl_data_module = SequenceDataModule(args)

Global seed set to 42


============================== PeptideDataModule __init__ Start ==============================
============================== PeptideDataModule __init__ End ==============================
CPU times: user 0 ns, sys: 4.07 ms, total: 4.07 ms
Wall time: 3.05 ms


In [6]:
%%time
pl_data_module.prepare_data('train')
pl_data_module.setup('fit')
pl_data_module.setup('test')

============================== prepare_data: mode[train] Start ==============================
============================== prepare_dataset Start ==============================
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< load data according to selected index >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


  0%|          | 0/2404 [00:00<?, ?it/s]

the dataset has not been partitioned, split dataset with specific ratio
dataframe partition values:
train    1661
test      521
valid     222
Name: partition, dtype: int64
select the subset for debug, max_len: 83, ratio: 1, number: 2404
============================== prepare_dataset End ==============================
len(self.train_dataset) 1661
len(self.valid_dataset) 222
len(self.test_dataset) 521
============================== prepare_data mode[train] End ==============================
============================== Setup [fit] Start ==============================
[len self.train_dataset] 1661
[len self.val_dataset] 222
============================== Setup [fit] End ==============================
============================== Setup [test] Start ==============================
[len self.test_dataset] 521
============================== Setup [test] End ==============================
CPU times: user 892 ms, sys: 373 ms, total: 1.27 s
Wall time: 1.31 s


In [7]:
pl_data_module.dataframe

,index,name,partition,length,sequence,structure,graph,dS,ddG,bins
0,0,0,train,83,STIEEQAKTFLDKFNHEAEDLFYQSSLASWNYNTNITEENVQNMNN...,None,None,0.017,-2.4635,"(-2.483, -2.457]"
1,1,1,train,83,STIEEQAKTFLDKFNHEAEDLFYQSSLASWNYNTNITEENVQNMNN...,None,None,0.017,-0.0984,"(-0.0997, -0.0964]"
2,10,10,train,83,STIEEQAKTFLDKFNHEAEDLFYQSSLARWNYNTNITEENVQNMNN...,None,None,0.017,-0.7424,"(-0.748, -0.742]"
3,100,100,train,83,STIEEQAKTFLDKFNHEAEDLFYQSSMAFWNYNTNITEENVQNMNN...,None,None,0.017,-1.1378,"(-1.14, -1.128]"
4,1000,1000,train,83,STIEEQAKTFLDKFNHEAEDLFYQSSLASWNYNTNITEENVQNDNN...,None,None,0.017,-1.3581,"(-1.359, -1.35]"
...,...,...,...,...,...,...,...,...,...,...
2399,995,995,test,83,SEIEEQAKTFLDKFNHEAEDLFYQSSLASWKYNTNITEENVQNMNN...,None,None,0.017,-0.0430,"(-0.0474, -0.0426]"
2400,996,996,test,83,SEIEEQAKTFLDKFNHEAEDLYYQSSLASWNYNTNITEENVQNMNN...,None,None,0.060,-1.1875,"(-1.189, -1.172]"
2401,997,997,test,83,STIEEQAKTFLDKFNHEAEDLFYQSSLASWNYNTNITEENVQNMNN...,None,None,0.017,-0.3152,"(-0.32, -0.31]"
2402,998,998,test,83,STIEEQAMTFLDKFKHEAEDLYYQSSLASWNYNTNITEENVQNMNN...,None,None,0.017,-1.0239,"(-1.029, -1.021]"


In [8]:
batch_tokens, batch_features, batch_ddG, batch_dS = next(iter(pl_data_module.train_dataloader()))
batch_tokens.shape, batch_features.shape, batch_ddG.shape, batch_dS.shape

(torch.Size([64, 85]),
 torch.Size([64, 1360]),
 torch.Size([64]),
 torch.Size([64]))

In [9]:
%%time
seed_everything(args.seed)
pl_model = SequenceLightningModule(args)

trainer = get_pl_trainer(args)

Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


CPU times: user 129 ms, sys: 36.4 ms, total: 166 ms
Wall time: 168 ms


In [10]:
pl_model.model

Net(
  (mlp): MLP(
    (mlp): Sequential(
      (0): Linear(in_features=1360, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=2, bias=True)
    )
  )
)

In [11]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
%%time
pl_data_module.prepare_data('train')
trainer.fit(model=pl_model, datamodule=pl_data_module)

Missing logger folder: ./script/task_04_LatentRegression/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type    | Params
-------------------------------------
0 | model    | Net     | 348 K 
1 | mse_loss | MSELoss | 0     
-------------------------------------
348 K     Trainable params
0         Non-trainable params
348 K     Total params
1.396     Total estimated model params size (MB)


============================== prepare_data: mode[train] Start ==============================
len(self.train_dataset) 1661
len(self.valid_dataset) 222
len(self.test_dataset) 521
============================== prepare_data mode[train] End ==============================
============================== prepare_data: mode[None] Start ==============================
============================== prepare_data has been invoked, mode=None ==============================
============================== prepare_data mode[None] End ==============================
============================== Setup [fit] Start ==============================
[len self.train_dataset] 1661
[len self.val_dataset] 222
============================== Setup [fit] End ==============================


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]